In [46]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [47]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/experiments")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [62]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Calculation
                OR n:StereologyDetails 
                OR n:Distributions 
                OR n:CellMorphology 
                OR n:Analysis
                OR n:Experiment
                OR n:SectioningDetail
                OR n:SectioningInstrument
                OR n:ReporterIncubation
                OR n:Reporter
                OR n:ReporterLabel
                OR n:ReporterTarget
                OR n:ReporterType
                OR n:VisualizationProtocol
                OR n:Solution
                OR n:Microscope
                OR n:ElectronMicroscopeDetail
                OR n:LightFluorescenceMicroscopeDetail
            DETACH DELETE n
        ''')
        print("wipeddatabase")

wipeddatabase


In [49]:
# Adding Calculation to graph from calculations.csv
csv_file_path = "file:///%s" % get_csv_path("calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Calculation { id: row.ID, description: row.Calculation_performed})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Calculation", session)

Added 958 nodes of type Calculation


In [50]:
# Adding StereologyDetails to graph from stereology_details.csv
csv_file_path = "file:///%s" % get_csv_path("stereology_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:StereologyDetails { id: row.ID, name: row.Stereology_details_name, probe: row.Probe, identificationFeature: row.Identification_feature, disectorHeight: toInt(row.Disector_height), areaSubfraction: toFloat(row.Area_subfraction), heightSubfraction: toFloat(row.Height_subfraction), investigatedSections: toFloat(row.Investigated_sections), investigatedFields: toInt(row.Investigated_fields), countedObjects: toInt(row.Counted_objects), coefficientOfError: toFloat(row.Coefficient_of_error), estimatedVolume: toFloat(row.Estimated_volume), volumeUnit: row.Volume_unit, anyExceptProbe: toInt(row.any_except_probe)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("StereologyDetails", session)

Added 212 nodes of type StereologyDetails


In [51]:
# Adding Distribution to graph from distributions.csv
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Distribution { id: row.ID, name: row.Distribution_name, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, distribution: row.Distribution, distributionDimensions: row.Distribution_dimensions, analysisTypePrimary: row.Analysis_type_primary, analysisTypeSecondary: row.Analysis_type_secondary})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Distribution", session)

Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2283)'")
Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2283)'")


Added 72 nodes of type Distribution


In [52]:
# Adding CellMorphology to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellMorphology { id: row.ID, name: row.Morphology_name, neuromorphoId: row.Neuromorpho_ID, somaSurface: toFloat(row.Soma_surface), numberOfStems: toInt(row.Number_stems), numberOfBifurcations: toInt(row.Number_bifurcations), numberOfBranches: toInt(row.Number_branches), overallWidth: toFloat(row.Overall_width), overallHeight: toFloat(row.Overall_height), overallDepth: toFloat(row.Overall_depth), averageBranchDiameter: toFloat(row.Average_branch_diameter), averageContraction: toFloat(row.Average_contraction), totalArborLength: toFloat(row.Total_arbor_length), totalArborSurface: toFloat(row.Total_arbor_surface), totalArborVolume: toFloat(row.Total_arbor_volume), maxEuclideanDistance: toFloat(row.Max_Euclidean_distance), maxPathDistance: toFloat(row.Max_path_distance), maxBranchOrder: toInt(row.Max_branch_order), totalFragmentation: toInt(row.Total_fragmentation), partitionAsymmetry: toFloat(row.Partition_asymmetry), averageRalls: toInt(row.Average_Ralls), averageBifurcationAngleLocal: toFloat(row.Average_bifurcation_angle_local), averageBifurcationAngleRemote: toFloat(row.Average_bifurcation_angle_remote), fractalDimension: toFloat(row.Fractal_dimension), physicalIntegrity: row.Physical_integrity, structuralDomains: row.Structural_domains, morphologicalAttributes: row.Morphological_attributes, originalFormat: row.Original_format})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("CellMorphology", session)

Added 50 nodes of type CellMorphology


In [53]:
# Adding Quantitation to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Quantitation { id: row.ID, name: row.Quantitation_name, estimateRelevance: row.Estimate_relevance, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, originalExtent: row.Original_extent, number: toInt(row.Number), numberSD: toInt(row.Number_SD), density: toFloat(row.Density), densityUnit: row.Density_unit, densitySD: toInt(row.Density_SD), volumetricDensity: toInt(row.Volumetric_density), estimateExtraction: row.Estimate_extraction})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Quantitation", session)

Added 3222 nodes of type Quantitation


In [54]:
# Adding Analysis to graph from derived_data_records.csv
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Analysis { id: row.ID, name: row.Derived_data_record_name, numberOfAnimals: row.Number_of_animals, shrinkageCorrection: row.Shrinkage_correction })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Analysis", session)

Added 674 nodes of type Analysis


In [55]:
# Adding Experiment to graph from experiments.csv
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Experiment { id: row.ID, name: row.Experiment_name, source: row.Source, animalStatus: row.Animal_status, ageLowerLimit: toInt(row.Age_lower_limit), ageUpperLimit: toInt(row.Age_upper_limit), weightLowerLimit: toInt(row.Weight_lower_limit), weightUpperLimit: toInt(row.Weight_upper_limit) })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Experiment", session)

Added 375 nodes of type Experiment


In [63]:
# Adding SectioningDetail to graph from experiments.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningDetail { id: row.ID, sectionThickness: toFloat(row.Section_thickness), sectionOrientation: row.Section_orientation })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningDetail", session)

Added 510 nodes of type SectioningDetail


In [68]:
# Adding SectioningInstrument to graph from sectioning_instruments.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_instruments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningInstrument { id: row.ID, name: row.Sectioning_instrument_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningInstrument", session)

Added 719 nodes of type SectioningInstrument


In [ ]:
# Adding ReporterIncubation to graph from reporter_incumations.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_incumations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterIncubation { id: row.ID, order: row.Order, concentration: row.Concentration, time: toFloat(row.Time), temperature: toInt(row.temperature) })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterIncubation", session)

In [70]:
# Adding Reporter to graph from reporters.csv
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Reporter { id: row.ID, name: row.Name, type: row.Reporter_type, uniqueId: row.Reporter_unique_ID, comment: row.Comment, originSpecie: row.Origin_species  })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Reporter", session)

Added 249 nodes of type Reporter


In [71]:
# Adding ReporterLabel to graph from reporter_labels.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_labels.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterLabel { id: row.ID, name: row.Label})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterLabel", session)

Added 23 nodes of type ReporterLabel


In [73]:
# Adding ReporterTarget to graph from reporter_targets.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_targets.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterTarget { id: row.ID, phenotype: row.Phenotype})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterTarget", session)

Added 123 nodes of type ReporterTarget


In [74]:
# Adding VisualizationProtocol to graph from visualization_protocols.csv
csv_file_path = "file:///%s" % get_csv_path("visualization_protocols.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:VisualizationProtocol { id: row.ID, name: row.Protocol_name, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("VisualizationProtocol", session)

Added 12 nodes of type VisualizationProtocol


In [75]:
# Adding Solution to graph from solutions.csv
csv_file_path = "file:///%s" % get_csv_path("solutions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Solution { id: row.ID, name: row.Solution_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Solution", session)

Added 266 nodes of type Solution


In [76]:
# Adding Microscope to graph from solutions.csv
csv_file_path = "file:///%s" % get_csv_path("microscopes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Microscope { id: row.ID, type: row.Microscope_type, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Microscope", session)

Added 9 nodes of type Microscope


In [66]:
## Reporter RELATIONS
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")

## :Reporter - :LABEL - :ReporterLabel
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterLabel { id: row.Label })
        MERGE (a)-[:LABEL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("LABEL", "Reporter", "ReporterLabel", session)
    
## :Reporter - :TARGET - :ReporterTarget
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterTarget { id: row.Target })
        MERGE (a)-[:TARGET]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET", "Reporter", "ReporterTarget", session)

Added 6 nodes of type ReporterIncubation


In [65]:
## SectioningDetail RELATIONS
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")

## :SectioningDetail - :SECTIONED_BY - :SectioningInstrument
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:SectioningDetail { id: row.ID})
        MATCH (c:SectioningInstrument { id: row.Sectioning_instrument })
        MERGE (a)-[:SECTIONED_BY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONED_BY", "SectioningDetail", "SectioningInstrument", session)


Added 382 relationships of type SectioningDetail SECTIONED_BY SectioningInstrument


In [ ]:
### Experiment RELATIONS
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")

## :Experiment - :ANAESTHETIC - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Anaesthetic })
        MERGE (a)-[:ANAESTHETIC]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ANAESTHETIC", "Experiment", "Solution", session)
    
## :Experiment - :PERFUSION_FIX_MEDIUM - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Perfusion_fix_medium })
        MERGE (a)-[:PERFUSION_FIX_MEDIUM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFUSION_FIX_MEDIUM", "Experiment", "Solution", session)



In [ ]:

### Experiment RELATIONS
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")

## :Experiment - :SPE - :Specimen
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Specimen { id: row.Specimen })
        MERGE (a)-[:SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN", "Analysis", "Specimen", session)

In [42]:
### Analysis RELATIONS
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")

## :Analysis - :SPECIMEN - :Specimen
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Specimen { id: row.Specimen })
        MERGE (a)-[:SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN", "Analysis", "Specimen", session)
    
## :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:NeuralStructure { id: row.Object_of_interest })
        MERGE (a)-[:OBJECT_OF_INTEREST]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBJECT_OF_INTEREST", "Analysis", "NeuralStructure", session)
    
## Add property to :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis)-[r:OBJECT_OF_INTEREST]-(b:NeuralStructure)
        WHERE row.OOI_recognition_criteria IS NOT NULL
        SET r.recognitionCriteria = row.OOI_recognition_criteria
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
## :Analysis - :CELL_TYPE_PUTATIVE - :Cell
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:Cell { id: row.Cell_type_putative })
        MERGE (a)-[:CELL_TYPE_PUTATIVE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_TYPE_PUTATIVE", "Analysis", "Cell", session)
    
## :Analysis - :VISUALIZATION_METHOD - :VisualizationMethod
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:VisualizationMethod { id: row.Visualization_method })
        MERGE (a)-[:VISUALIZATION_METHOD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("VISUALIZATION_METHOD", "Analysis", "VisualizationMethod", session)


Added 0 relationships of type Analysis SPECIMEN Specimen
Added 668 relationships of type Analysis OBJECT_OF_INTEREST NeuralStructure
Added 668 relationships of type Analysis CELL_TYPE_PUTATIVE Cell
Added 0 relationships of type Analysis VISUALIZATION_METHOD VisualizationMethod


In [ ]:
### Quantitation RELATIONS

## :Analysis - :DATA_TYPE - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Quantitation { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Quantitation", session)
    
## :Quantitation - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Quantitation", "RegionRecord", session)
    
## :Quantitation - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "Quantitation", "RegionZone", session)
    
## :Quantitation - :CELLULAR_TARGET_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_target_region })
        MERGE (a)-[:CELLULAR_TARGET_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_TARGET_REGION", "Quantitation", "CellularRegion", session)
    
## :Quantitation - :TARGET_CELL - :Cell
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Cell { id: row.Cellular_target_ID })
        MERGE (a)-[:TARGET_CELL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET_CELL", "Quantitation", "Cell", session)

## :Quantitation - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Quantitation", "StereologyDetail", session)
    
## :Quantitation - :SOFTWARE - :Software
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:SOFTWARE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SOFTWARE", "Quantitation", "Software", session)

In [ ]:
### CellMorphology RELATIONS

## :Analysis - :DATA_TYPE - :CellMorphology
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:CellMorphology { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "CellMorphology", session)

## :CellMorphology - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "CellMorphology", "RegionRecord", session)
    
## :CellMorphology - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "CellMorphology", "RegionZone", session)
    
## :CellMorphology - :RECONSTRUCTED_WITH - :Software
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:Software { id: row.Reconstruction_method })
        MERGE (a)-[:RECONSTRUCTED_WITH]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RECONSTRUCTED_WITH", "CellMorphology", "Software", session)

In [ ]:
### DISTRIBUTION RELATIONS

## :Analysis - :DATA_TYPE - :Distribution
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Distribution { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Distribution", session)
    
## :Distribution - :CELLULAR_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_region })
        MERGE (a)-[:CELLULAR_REGION {cytochemicalId: row.Cellular_region_cytochemical_ID}]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_REGION", "Distribution", "CellularRegion", session)

## :Distribution - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Distribution", "RegionRecord", session)
    
## :Distribution - :PERFORMED_ON - :Software
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:PERFORMED_ON]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFORMED_ON", "Distribution", "Software", session)

## :Distribution - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Distribution", "StereologyDetail", session)
    
## :Distribution - :RELATED_TO - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Quantitation { id: row.Related_quantitation })
        MERGE (a)-[:RELATED_TO]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RELATED_TO", "Distribution", "Quantitation", session)

